# W266 Project

### Adam Sayre & Erin Werner

## Single Layer Perceptron Model

In [1]:
import numpy as np
import csv
import pandas as pd 
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import importlib
import emoji
import tensorflow as tf
import nltk
import re
from nltk.corpus import brown
nltk.download('stopwords')
from nltk.corpus import stopwords
assert(nltk.download("treebank"))
from nltk.corpus import europarl_raw
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erinwerner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/erinwerner/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [2]:
# Keras libraries
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow.keras

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [4]:
data = pd.read_csv("~/Downloads/dataset(clean).csv") 
data.head()

,Emotion,Content,Original Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t..."
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...


In [5]:
data_e = pd.read_csv("~/Downloads/dataset(clean)_e.csv") 
data_e.head()[['Emotion','Content','Original Content','E_Content']]

,Emotion,Content,Original Content,E_Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...,rt usertaginstance usertaginstance oh fuck wro...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it,feel shamed
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...,feeling little bit defeated steps faith would ...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t...",usertaginstance imagine reaction guy called jj...
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...,wouldnt feel burdened would live life testamen...


In [6]:
data_a = pd.read_csv("~/Downloads/dataset(clean)_a.csv") 
data_a.head()[['Emotion','Content','Original Content','A_Content']]

,Emotion,Content,Original Content,A_Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...,b rt davbingodav mcrackins oh fuck wrote fil g...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it,feel shamed
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...,feeling little bit defeated steps faith would ...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t...",b ksiolajidebt imagine reaction guy called jj ...
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...,wouldnt feel burdened would live life testamen...


We can build the model, then pass each of our various cleaning methods through it.

In [7]:
# Some starting variables
vocab_size = 10000
max_length = 40

In [8]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(vocab_size, 8, input_length=max_length, embeddings_regularizer='l1'))
emb_model.add(layers.Flatten())
emb_model.add(layers.Dense(3, activation='relu'))
emb_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [9]:
emb_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 320)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 963       
Total params: 80,963
Trainable params: 80,963
Non-trainable params: 0
_________________________________________________________________


#### Original Cleaned Data

In [10]:
possible_labels = data.Emotion.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'disappointed': 0, 'happy': 1, 'angry': 2}

In [11]:
data['label'] = data.Emotion.replace(label_dict)

In [12]:
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(data.Content.values, data.label.values, test_size=0.3, 
                                                  random_state=42, stratify=data.label.values)

In [13]:
tokenizer_c = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower=True, split = " ")
tokenizer_c.fit_on_texts(X_train_c)

X_train_seq_c = tokenizer_c.texts_to_sequences(X_train_c)
X_train_seq_trunc_c = pad_sequences(X_train_seq_c, maxlen=max_length)

X_val_seq_c = tokenizer_c.texts_to_sequences(X_val_c)
X_val_seq_trunc_c = pad_sequences(X_val_seq_c, maxlen=max_length)

In [14]:
# Encoding output variable
le_c = LabelEncoder()

y_train_le_c = le_c.fit_transform(y_train_c)
y_train_emb_c = to_categorical(y_train_le_c)

y_val_le_c = le_c.transform(y_val_c)
y_val_emb_c = to_categorical(y_val_le_c)

In [15]:
emb_model.reset_states()
emb_model.fit(X_train_seq_trunc_c, y_train_emb_c, epochs=5)

Epoch 1/5
20051/20051 [==============================] - 17s 805us/step - loss: nan - accuracy: 0.3420
Epoch 2/5
20051/20051 [==============================] - 16s 775us/step - loss: nan - accuracy: 0.3420
Epoch 3/5
20051/20051 [==============================] - 16s 777us/step - loss: nan - accuracy: 0.3425
Epoch 4/5
20051/20051 [==============================] - 16s 790us/step - loss: nan - accuracy: 0.3419
Epoch 5/5
20051/20051 [==============================] - 16s 782us/step - loss: nan - accuracy: 0.3421


In [16]:
percp_loss_c, percp_accuracy_c = emb_model.evaluate(X_train_seq_trunc_c, y_train_emb_c, verbose=False)
print("Training Accuracy: {:.4f}".format(percp_accuracy_c))
percp_loss_c_val, percp_accuracy_c_val = emb_model.evaluate(X_val_seq_trunc_c, y_val_emb_c, verbose=False)
print("Validation Accuracy:  {:.4f}".format(percp_accuracy_c_val))

Training Accuracy: 0.3423
Validation Accuracy:  0.3423


#### Original Uncleaned Data

In [17]:
data["Original_Content"] = data["Original Content"]

In [18]:
X_train_oc, X_val_oc, y_train_oc, y_val_oc = train_test_split(data.Original_Content.values, data.label.values, 
                                                  test_size=0.3, random_state=42, stratify = data.label.values)

In [19]:
tokenizer_oc = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower=True, split = " ")
tokenizer_oc.fit_on_texts(X_train_oc)

X_train_seq_oc = tokenizer_oc.texts_to_sequences(X_train_oc)
X_train_seq_trunc_oc = pad_sequences(X_train_seq_oc, maxlen=max_length)

X_val_seq_oc = tokenizer_oc.texts_to_sequences(X_val_oc)
X_val_seq_trunc_oc = pad_sequences(X_val_seq_oc, maxlen=max_length)

In [20]:
# Encoding output variable
le_oc = LabelEncoder()

y_train_le_oc = le_oc.fit_transform(y_train_oc)
y_train_emb_oc = to_categorical(y_train_le_oc)

y_val_le_oc = le_oc.transform(y_val_oc)
y_val_emb_oc = to_categorical(y_val_le_oc)

In [21]:
emb_model.reset_states()
emb_model.fit(X_train_seq_trunc_oc, y_train_emb_oc, epochs=5)

Epoch 1/5
20051/20051 [==============================] - 16s 802us/step - loss: nan - accuracy: 0.3423
Epoch 2/5
20051/20051 [==============================] - 16s 795us/step - loss: nan - accuracy: 0.3423
Epoch 3/5
20051/20051 [==============================] - 15s 752us/step - loss: nan - accuracy: 0.3423
Epoch 4/5
20051/20051 [==============================] - 15s 766us/step - loss: nan - accuracy: 0.3423
Epoch 5/5
20051/20051 [==============================] - 15s 761us/step - loss: nan - accuracy: 0.3423


In [22]:
percp_loss_oc, percp_accuracy_oc = emb_model.evaluate(X_train_seq_trunc_oc, y_train_emb_oc, verbose=False)
print("Training Accuracy: {:.4f}".format(percp_accuracy_oc))
percp_loss_oc_val, percp_accuracy_oc_val = emb_model.evaluate(X_val_seq_trunc_oc, y_val_emb_oc, verbose=False)
print("Validation Accuracy:  {:.4f}".format(percp_accuracy_oc_val))

Training Accuracy: 0.3423
Validation Accuracy:  0.3423


#### Custom Cleaned Data #1

In [23]:
possible_labels_e = data_e.Emotion.unique()

label_dict_e = {}
for index, possible_label in enumerate(possible_labels_e):
    label_dict_e[possible_label] = index
label_dict_e

{'disappointed': 0, 'happy': 1, 'angry': 2}

In [24]:
data_e['label'] = data_e.Emotion.replace(label_dict_e)

In [25]:
X_train_e, X_val_e, y_train_e, y_val_e = train_test_split(data_e.E_Content.values, data_e.label.values, 
                                                  test_size=0.30,random_state=42, stratify=data_e.label.values)

In [26]:
tokenizer_e = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower=True, split = " ")
tokenizer_e.fit_on_texts(X_train_e)

X_train_seq_e = tokenizer_e.texts_to_sequences(X_train_e)
X_train_seq_trunc_e = pad_sequences(X_train_seq_e, maxlen=max_length)

X_val_seq_e = tokenizer_e.texts_to_sequences(X_val_e)
X_val_seq_trunc_e = pad_sequences(X_val_seq_e, maxlen=max_length)

In [27]:
# Encoding output variable
le_e = LabelEncoder()

y_train_le_e = le_e.fit_transform(y_train_e)
y_train_emb_e = to_categorical(y_train_le_e)

y_val_le_e = le_e.transform(y_val_e)
y_val_emb_e = to_categorical(y_val_le_e)

In [29]:
emb_model.reset_states()
emb_model.fit(X_train_seq_trunc_e, y_train_emb_e, epochs=5)

Epoch 1/5
20051/20051 [==============================] - 16s 797us/step - loss: nan - accuracy: 0.3423
Epoch 2/5
20051/20051 [==============================] - 16s 776us/step - loss: nan - accuracy: 0.3423
Epoch 3/5
20051/20051 [==============================] - 16s 786us/step - loss: nan - accuracy: 0.3423
Epoch 4/5
20051/20051 [==============================] - 15s 766us/step - loss: nan - accuracy: 0.3423
Epoch 5/5
20051/20051 [==============================] - 15s 758us/step - loss: nan - accuracy: 0.3423


In [30]:
percp_loss_e, percp_accuracy_e = emb_model.evaluate(X_train_seq_trunc_e, y_train_emb_e, verbose=False)
print("Training Accuracy: {:.4f}".format(percp_accuracy_e))
percp_loss_e_val, percp_accuracy_e_val = emb_model.evaluate(X_val_seq_trunc_e, y_val_emb_e, verbose=False)
print("Validation Accuracy:  {:.4f}".format(percp_accuracy_e_val))

Training Accuracy: 0.3423
Validation Accuracy:  0.3423


#### Custom Cleaned Data #2

In [31]:
#read from dataframe
possible_labels_a = data_a.Emotion.unique()

label_dict_a = {}
for index, possible_label in enumerate(possible_labels_a):
    label_dict_a[possible_label] = index
label_dict_a
data_a['label'] = data_a.Emotion.replace(label_dict_a)

In [32]:
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(data_a.A_Content.values, data_a.label.values, 
                                                  test_size=0.30,random_state=42, stratify=data_a.label.values)

In [33]:
tokenizer_a = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower=True, split = " ")
tokenizer_a.fit_on_texts(X_train_a)

X_train_seq_a = tokenizer_a.texts_to_sequences(X_train_a)
X_train_seq_trunc_a = pad_sequences(X_train_seq_a, maxlen=max_length)

X_val_seq_a = tokenizer_a.texts_to_sequences(X_val_a)
X_val_seq_trunc_a = pad_sequences(X_val_seq_a, maxlen=max_length)

In [34]:
# Encoding output variable
le_a = LabelEncoder()

y_train_le_a = le_a.fit_transform(y_train_a)
y_train_emb_a = to_categorical(y_train_le_a)

y_val_le_a = le_e.transform(y_val_a)
y_val_emb_a = to_categorical(y_val_le_a)

In [35]:
emb_model.reset_states()
emb_model.fit(X_train_seq_trunc_a, y_train_emb_a, epochs=5)

Epoch 1/5
20051/20051 [==============================] - 16s 804us/step - loss: nan - accuracy: 0.3423
Epoch 2/5
20051/20051 [==============================] - 16s 781us/step - loss: nan - accuracy: 0.3423
Epoch 3/5
20051/20051 [==============================] - 16s 779us/step - loss: nan - accuracy: 0.3423
Epoch 4/5
20051/20051 [==============================] - 15s 762us/step - loss: nan - accuracy: 0.3423
Epoch 5/5
20051/20051 [==============================] - 15s 759us/step - loss: nan - accuracy: 0.3423


In [36]:
percp_loss_a, percp_accuracy_a = emb_model.evaluate(X_train_seq_trunc_a, y_train_emb_a, verbose=False)
print("Training Accuracy: {:.4f}".format(percp_accuracy_a))
percp_loss_a_val, percp_accuracy_a_val = emb_model.evaluate(X_val_seq_trunc_a, y_val_emb_a, verbose=False)
print("Validation Accuracy:  {:.4f}".format(percp_accuracy_a_val))

Training Accuracy: 0.3423
Validation Accuracy:  0.3423


#### Results

In [38]:
p_acc = [percp_accuracy_oc, percp_accuracy_c, percp_accuracy_e, percp_accuracy_a]
#p_loss = [percp_loss_oc, percp_loss_c, percp_loss_e, percp_loss_a]
p_values = ['Orig. Uncleaned', 'Orig. Cleaned', 'Custom Cleaned #1', 'Custom Cleaned #2']
p_df = pd.DataFrame()
p_df['Cleaning Method'] = p_values
p_df['F1 Score'] = p_acc
#p_df['Loss'] = p_loss
p_df

,Cleaning Method,F1 Score
0,Orig. Uncleaned,0.342268
1,Orig. Cleaned,0.342268
2,Custom Cleaned #1,0.342268
3,Custom Cleaned #2,0.342268
